In [1]:
#!/usr/bin/python
# Modified 11/29/2019 by HYK
# classical gene activation model
# Incorporate binomial partitioning, DONE!
# Cumulative addition of time over multiple rounds of simulation, DONE!

# Generalized functions for simulating arbitrary reaction network topologies
# Simulation cell number distributions as well, for multiple runs
# Also, write a MATLAB script to rapidly generate plots from the simulated data

"""
Created on Tue Dec 13 20:08:54 2016
@author: Sam Nguyen
"""
import roadrunner
import numpy 
from numpy import random as rand
import math
from scipy import asarray as ar,exp
from multiprocessing import Pool
import csv
import random
import sys
roadrunner.Config.setValue(roadrunner.Config.MAX_OUTPUT_ROWS,10000)

# Tellurium loader
def loada(sbstr):
    import antimony as sb
    r = sb.loadAntimonyString(sbstr)
    if r < 0:
        raise RuntimeError('Failed to load Antimony model: {}'.format(sb.getLastError()))
    return roadrunner.RoadRunner(sb.getSBMLString(sb.getModuleNames()[-1]))

# Tellurium simulation function
def doSimulation(tc,nc):
    r = loada("""
    # define cell cycling parameters
    TCycle = 10                                    
    V := (time/TCycle + 1)
    
    #define ode    
    R1: => X ; S*alphaX*V        # synthesis of X, rate increases with V
    R2: X => ; X*deltaX             # degradation of X
    R3: => Y ; (alphaY*V)*(X/V)^N/((X/V)^N+K^N)
    R4: Y => ; Y*deltaY
   
   #define initial values
    S = 200          # the concentration of signal * (normalized units) 
    X = 0              # the initial number of copies of X
    Y = 0              # the initial number of copies of Y
 
    alphaX = 0.5          # synthesis rate of X (copies/hr)
    deltaX = 0             # degradation rate (1/hr), assume infinitely stable
    alphaY = 10^3          # synthesis rate of Y (copies/hr)
    deltaY = 1             # degradation rate (1/hr)
    K = 1000              # MM constant for activation of Y by X
    N = 10                    # Hill coefficient for activation of Y by X, switch like response (unitless)  
    """)

    # set parameters of stochastic simulation
    r.integrator = 'gillespie'
    r.integrator.variable_step_size = False  
    r.TCycle = tc # define cell cycle time 
    
    ############################
    dt = 0.1;     # time interval for plotting
    result=[]
    for i in range(0,nc,1):       # for loop over the total number of cell cycles
        if i == 0:
            SubResult =r.simulate(0,tc,int(tc/dt),['time','V','X','Y'])
            for row in SubResult:
                result.append(row)
        else:
            r.X = rand.binomial(r.X, 0.5)   # randomly partition
            r.Y = rand.binomial(r.Y, 0.5)
            SubResult =r.simulate(0,tc,int(tc/dt),['time','V','X','Y'])             
            for row in SubResult:
                row[0]=row[0]+(i*tc)     # update the time vector
                result.append(row)
    result = ar(result)
    return result

#Write csv file
def writeCSV(filename,result):
    with open(filename,'w') as mycvsfile:
        thedatawriter=csv.writer(mycvsfile)   
        thedatawriter.writerows(result)


In [2]:
for i in range(0,100,1):
    result = doSimulation(10,25)
    #writeCSV('data/data_{}.csv'.format(i),result)



C:\Users\samng\Anaconda3\lib\site-packages\ipykernel_launcher.py:80: DeprecationWarning: scipy.asarray is deprecated and will be removed in SciPy 2.0.0, use numpy.asarray instead
